<a href="https://colab.research.google.com/github/lohansilva/analise-covid19/blob/main/analise_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install selenium
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.tR7dZQfkOQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.SFxEcuOyr0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.pKFpL5elwi/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [26]:
!apt-get update
!apt-get install chromium chromium-driver

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,554 kB]
Hit:12 http://archive.ubuntu.com/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:14 http://archive.ubuntu.com/ubuntu focal-backports In

In [27]:
from selenium import webdriver

In [28]:
#importando as bibliotecas necessárias para leitura dos dados, filtragem e exportação dos dados em excel
import requests
import pandas as pd
import xlsxwriter

In [29]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [30]:
driver = web_driver()

In [31]:
driver.get('https://ourworldindata.org/covid-cases')

In [32]:
botao_download = driver.find_element("xpath", "/html/body/main/article/div[3]/div[2]/div/div/section[1]/div[2]/div[1]/div/div/div/a[2]")

In [33]:
botao_download.click()

In [34]:
# Fecha o navegador
driver.quit()

In [35]:
# Cria um DataFrame a partir dos dados extraídos
covid_data = pd.read_csv('/content/owid-covid-data.csv.crdownload')

In [36]:
# Filtrar dados do Brasil e em um determinado periodo
brasil_data = covid_data[(covid_data['location'] == 'Brazil') & (covid_data['date'] >= '2023-01-01') & (covid_data['date'] <= '2023-01-05')]

In [37]:
# limpar colunas, para deixar apenas as importantes para análise
brasil_data = brasil_data[['date', 'new_cases', 'new_deaths']]

In [38]:
# Criar novo arquivo Excel
workbook = xlsxwriter.Workbook('analise_covid.xlsx')

In [39]:
# se tiver dados nan, alterar para o valor "0", para evitar erros
brasil_data = brasil_data.fillna(0)

In [40]:
#criar planilha com os dados
worksheet = workbook.add_worksheet('Dados')
worksheet.write_row(0, 0, brasil_data.columns)
for i, row in enumerate(brasil_data.values):
    worksheet.write_row(i+1, 0, row)

In [41]:
# Adicionar gráfico de linhas sobre as novas mortes por covid 
chart = workbook.add_chart({'type': 'line'})
chart.add_series({
    'categories': ['Dados', 1, 0, len(brasil_data), 0],
    'values': ['Dados', 1, 2, len(brasil_data), 2],
    'name': 'Mortes Confirmadas'
})

chart.set_title({'name': 'Evolução de mortes por COVID-19 no Brasil(5 dias aleatorios)'})
chart.set_x_axis({'name': 'Data'})
chart.set_y_axis({'name': 'Número de novas mortes'})
worksheet.insert_chart('F2', chart)

0

In [42]:
# Adicionar gráfico de linhas sobre os novos casos confirmados
chart2 = workbook.add_chart({'type': 'line'})
chart2.set_y_axis({'name': 'Casos confirmados','min': 0, 'max': 35000})
chart2.add_series({
    'categories': ['Dados', 1, 0, len(brasil_data), 0],
    'values': ['Dados', 1, 1, len(brasil_data), 1],
    'name': 'Casos confirmados'
})
chart2.set_title({'name': 'Evolução de novos casos confirmados por COVID-19 no Brasil(5 dias aleatorios)'})
chart2.set_x_axis({'name': 'Data'})
chart2.set_y_axis({'name': 'Número de casos'})
worksheet.insert_chart('N2', chart2)

0

In [43]:
# Salvar arquivo
workbook.close()